In [ ]:
import itertools
from itertools import groupby
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
import math
import copy

In [ ]:
class DataProcessor(object):
    def __init__(self):
        self.client = MongoClient('localhost', port=27017)
        self.db = self.client.train
        self.train = self.db.Train
        self.station = self.db.Station 

        self.train_df = pd.DataFrame(self.train.find({}, {'_id': 0}))
        self.train_df = self.train_df.drop_duplicates(subset='No', keep='first', inplace=False)

        self.station_df = pd.DataFrame(self.station.find({}, {'_id': 0}))
        self.station_df = self.station_df.drop_duplicates(subset='name', keep='first', inplace=False)

        self.mergedTrainLs = [] 
        self.adjacency_df = []
                          
    def addTrainsOfStation(self):
        """
        功能:将各个站点的经停列车加入Station数据库
        """
        station = self.train_df.key.tolist()
        passby = list(zip(self.train_df.code, self.train_df.key))
        # 添加经过车站的列车
        trainsOfStation = {}
        for (code, names) in passby:
            for name in names:
                if name in trainsOfStation:
                    trainsOfStation[name].append(code)
                elif name != "":
                    trainsOfStation[name] = []
        for (name, codes) in trainsOfStation.items():
            #print(name, codes)              
            query = {"name":name}
            newvalues = { "$set": { "trains": codes } }
            self.station.update_one(query, newvalues)

    def generateMergedTrain(self):
    '''
    功能:将运行在同一线路上的不同列车合并，存入self.mergedTrainLs
    依赖:self.train_df
    输出:self.mergedTrainLs
    '''
        # 起止站作为第一特征，以距离作为第二特征量
        documentsDict = {}
        #counter = 0
        deleteLs = []
        for index,document in self.train_df.iterrows():
            start_s = document["start_s"]
            end_s = document["end_s"]
            km = document["km"]
            key1 = f"{start_s}-{end_s}-{km}"
            key2 = f"{end_s}-{start_s}-{km}"
            # counter += 1
            # if counter > 10:
            #     break
            # 起止有问题的不要
            if start_s == "" or end_s == "":
                #deleteLs.append(key1)
                continue
            # 分正向与反向两种情况讨论
            if key1 in documentsDict:
                documentsDict[key1]["forward"].append(document["code"])
            elif key2 in documentsDict:
                documentsDict[key2]["backward"].append(document["code"])
            else:
                documentsDict[key1] = {"start_s":start_s, "end_s":end_s, "type":document["type"],"distance":[], "time":[], "stations":[], "forward": [document["code"]], "backward":[]}

                prev = [0, 0, 0, 0, 0, 0, "0:00"]
                for item in document["info"]:
                    if item[1] == "": #筛去空站
                        continue
                    prev_t = prev[6].split(':')
                    prev_t = int(prev_t[0])*60+int(prev_t[1])
                    cur_t = item[6].split(':')
                    cur_t = int(cur_t[0])*60+int(cur_t[1])
                    # 出现时光倒流的不要
                    if cur_t < prev_t or int(item[5])-int(prev[5]) < 0:
                        documentsDict.pop(key1)
                        deleteLs.append(key1)
                        break
                    documentsDict[key1]["stations"].append(item[1])
                    documentsDict[key1]["distance"].append(int(item[5])-int(prev[5]))
                    documentsDict[key1]["time"].append(cur_t-prev_t)
                    prev = item
        self.mergedTrainLs = list(documentsDict.values())
        print("舍弃的列车:", len(deleteLs))
        print(deleteLs)


    def generateAdjacencyList(self):
    '''
    功能:构建邻接矩阵
    依赖:self.station_df self.train_df self.mergedTrainLs
    输出:self.adjacency_df
    '''
        countnum = 200
        stationLs = self.station_df.name.tolist()
        # 建立站-值对应表
        stationValue = {}
        for index,document in self.station_df.iterrows():
            if document["trains"] is np.nan: #nan是个大坑, 以及没有车经过的站居然存在
                stationValue[document["name"]] = -1
            elif len(document["trains"]) == 0:
                stationValue[document["name"]] = -1
            else:
                stationValue[document["name"]] = math.log(len(document["trains"]))
        stationLs.sort(key=lambda x:stationValue[x], reverse=True)
        stationLs = stationLs[:countnum]
        #print(stationLs[:10])
        # 分别为时间，距离，连接强度
        d = {col:pd.Series([[1e10, 1e10, 0]]*len(stationLs), index=stationLs) for col in stationLs}
        self.adjacency_df = pd.DataFrame(d)
        print("Finish initialization")
        #print(pd.DataFrame(d))
        for item in self.mergedTrainLs:
            #print()
            stations = item["stations"]
            pre_time = []  #时间前缀和
            pre_dis =[]    #距离前缀和
            pre = 0
            for it in item["time"]:
                pre_time.append(it+pre)
                pre = it+pre
            pre = 0
            for it in item["distance"]:
                pre_dis.append(it+pre)
                pre = it+pre
            # 更新任意两站间的关系
            for i in range(len(stations)):
                if not (stations[i] in stationLs):
                    continue
                self.adjacency_df[stations[i]][stations[i]]=[0, 0, 0]
                for j in range(i+1, len(stations)):
                    if not (stations[j] in stationLs):
                        continue
                    ls = self.adjacency_df[stations[i]][stations[j]].copy()
                    # 更新时间依据
                    if ls[0] > pre_time[j]-pre_time[i]:
                        ls[0] = pre_time[j]-pre_time[i]
                        #ls[3] = item["type"]
                    # 更新空间依据
                    if ls[1] > pre_dis[j]-pre_dis[i]:
                        ls[1] = pre_dis[j]-pre_dis[i]
                        #ls[4] = item["type"]
                    ls[2] += (len(item["forward"])+len(item["backward"]))
                    self.adjacency_df[stations[i]][stations[j]] = ls.copy()
                    self.adjacency_df[stations[j]][stations[i]] = ls.copy()
            #print(item["start_s"]+item["end_s"]+"ok")
    
    def calAccess(self):
    '''
    功能:计算站点可达性
    依赖:self.train_df  self.station_df
    输出:./AccessInfo.json
    '''
        # 建立站-值对应表
        stationValue = {}
        for index,document in self.station_df.iterrows():
            if document["trains"] is np.nan: #nan是个大坑, 以及没有车经过的站居然存在
                stationValue[document["name"]] = -1
            elif len(document["trains"]) == 0:
                stationValue[document["name"]] = -1
            else:
                stationValue[document["name"]] = math.log(len(document["trains"]))

        # 计算每条路线带来的可达性
        trainAccess = {}   # 每条线的可达性
        passby = list(zip(self.train_df.code, self.train_df.key))
        for code, stations in passby:
            value = 0
            for station in stations:
                if station in stationValue:
                    value += stationValue[station]
            trainAccess[code] = value
        #print(trainAccess.items())
        
        # 计算每个站点的可达性
        stationAccess = {}
        for index,document in self.station_df.iterrows():
            if document["trains"] is np.nan: #nan是个大坑, 以及没有车经过的站居然存在
                stationAccess[document["name"]] = -1
            elif len(document["trains"]) == 0:
                stationAccess[document["name"]] = -1
            else:
                avalue = 0
                for code in document["trains"]:
                    if code in trainAccess:
                        avalue+= trainAccess[code]
                    # key = self.train_df[self.train_df["code"] == code].key
                    # if len(key) > 0:
                    #     key[0]
                stationAccess[document["name"]] = avalue

        # 构造可达性信息json文件
        with open("AccessInfo.json", 'w', encoding ="utf-8") as fw:
            fw.write(json.dumps(stationAccess, ensure_ascii=False))
    
    def Chart(self):
    '''
    功能: 生成前200个车站的关系信息
    依赖: ./min_adjacency_table.json
    输出: ./RelationChartInfo.json
    '''
        countnum = 50
        provinestr = "北京市、天津市、上海市、重庆市、河北省、山西省、辽宁省、吉林省、黑龙江省、江苏省、浙江省、安徽省、福建省、江西省、山东省、河南省、湖北省、湖南省、广东省、海南省、四川省、贵州省、云南省、陕西省、甘肃省、青海省、内蒙古自治区、广西壮族自治区、西藏自治区、宁夏回族自治区、新疆维吾尔自治区、香港特别行政区"
        ls = provinestr.split("、")
        ls = ["北京市", "天津市", "河北省", "上海市", "江苏省", "浙江省", "福建省", "山东省", "广东省", "海南省", "香港特别行政区"]
        provinceDict = {ls[i]:0 for i in range(len(ls))}
        ls = ["山西省", "安徽省", "江西省", "河南省", "湖北省", "湖南省"]
        for it in ls:
            provinceDict[it] = 1
        ls = ["内蒙古自治区","广西壮族自治区","重庆市","四川省","贵州省","云南省","西藏自治区","陕西省","甘肃省","青海省","宁夏回族自治区", "新疆维吾尔自治区"]
        for it in ls:
            provinceDict[it] = 2     
        ls = ["辽宁省", "吉林省", "黑龙江省"]
        for it in ls:
            provinceDict[it] = 3  
        ls = ["东部", "中部", "西部", "东北"]
        categories = [{"name":it} for it in ls]
        res = {"nodes":[], "edges":[], "categories":categories}
        # 生成节点信息
        ls = []
        for index,document in self.station_df.iterrows():
            name = document["name"]
            value = 0
            if document["trains"] is np.nan: #nan是个大坑, 以及没有车经过的站居然存在
                value = 0
            elif len(document["trains"]) == 0:
                value = 0
            else:
                value = math.log(len(document["trains"]))
            ls.append({"name":name, "value":value, 'category':provinceDict[document["province"]]})
        ls.sort(key=lambda x:x["value"], reverse=True)
        res["nodes"] = copy.deepcopy(ls)[:countnum]

        print("edge")
        # 生成边的关系
        df= pd.read_json("min_adjacency_table.json")
        #df = self.adjacency_df
        print("read ended")
        count = 0
        for index,document in df.iterrows():
            for item in dict(document).items():
                if index == item[0]:
                    break
                num = int(item[1][2])
                if num > 1:
                    num = math.ceil(math.log(num,2))
                for i in range(num):
                    res["edges"].append({"source":index, "target":item[0]})
            count += 1
            print(index, count)
            if count >= countnum:
                break

        with open("RelationChartInfo.json", 'w', encoding ="utf-8") as fw:
            fw.write(json.dumps(res, ensure_ascii=False))
    
# 实例化数据处理器
processor = DataProcessor()

In [ ]:
# !危险操作! 请保证此操作在数据构建过程中仅进行一次 向Station数据库加入车站经停列车信息
processor.addTrainsOfStation()

In [ ]:
# 将同一线路的列车合并,存入self.mergedTrainLs
processor.generateMergedTrain()

In [ ]:
# 构造新的列车信息json文件
with open("MergedTrainInfo.json", 'w', encoding ="utf-8") as fw:
    fw.write(json.dumps(processor.mergedTrainLs, ensure_ascii=False))

In [ ]:
# 计算站点可达性,存为json
processor.calAccess()

In [ ]:
# 生成前200个车站的关系信息,存为json
processor.Chart()

In [ ]:
# 构建邻接矩阵
processor.generateAdjacencyList()

In [ ]:
# 测试邻接矩阵构造情况
print(processor.adjacency_df["北京南"]["北京西"])

In [ ]:
# 储存邻接矩阵
with open('min_adjacency_table.json', 'w', encoding='utf-8') as file:
    processor.adjacency_df.to_json(file, force_ascii=False)

In [ ]:
# 测试邻接矩阵读取
ff= pd.read_json("min_adjacency_table.json")
print(ff["北京南"]["北京西"])